In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from openfast_toolbox.fastfarm.FASTFarmCaseCreation import FFCaseCreation

In [ ]:
# Pre-requisites
# - ROSCO libdiscon shared object (ROSCO build)

# Notes
# - Note how some inputs (ED, SD, turbine fst) are giving with the string ending in `*.T` and the actual filename inside `templatePath` is `*.T.<ext>` where `<ext>` is either `dat` or `fst`.
# - I will explain directory structure on the slides


# Example 1: `Ex1_FASTFarm_discretization.py`

This example calculates the desired temporal and spatial resolution given wind farm. The next two cells are exactly what is on the python script. I run this a few times as described below.

In [ ]:
# -----------------------------------------------------------------------------
# USER INPUT: Modify these
# -----------------------------------------------------------------------------

# -----------------------------------------------------------------------------
# ------------------------ General parameters ---------------------------------
# -----------------------------------------------------------------------------

# ----------- Case absolute path
path = '/projects/tcwnd/rthedin/fastfarm/ff_workshop_nawea2025/ff_examples_ex1'

# ----------- Execution parameters
# If you are sure the correct binaries are first on your $PATH and associated
# libraries on $LD_LIBRARY_PATH, you can set the variables below to None or
# remove them from the `FFCaseCreation` call
ffbin = '/projects/tcwnd/rthedin/repos/openfast_v4.1.1/build/install/bin/FAST.Farm'
tsbin = '/projects/tcwnd/rthedin/repos/openfast_v4.1.1/build/install/bin/turbsim'


# -----------------------------------------------------------------------------
# --------------------------- Farm parameters ---------------------------------
# -----------------------------------------------------------------------------

# ----------- General turbine parameters
cmax     = 5      # Maximum blade chord (m)
fmax     = 10/6   # Maximum excitation frequency (Hz)
Cmeander = 1.9    # Meandering constant (-)
D = 240           # Rotor diameter (m)
zhub = 150        # Hub height (m)

# ----------- Wind farm
# The wts dictionary holds information of each wind turbine. The allowed entries
# are: x, y, z, D, zhub, cmax, fmax, Cmeander, and phi_deg. The phi_deg is the
# only entry that is optional and is related to floating platform heading angle,
# given in degrees. The angle phi_deg is not illustrated on the example below.
wts = {
    0 :{'x': 500.0, 'y': 480, 'z':0.0, 'D':D, 'zhub':zhub, 'cmax':cmax, 'fmax':fmax, 'Cmeander':Cmeander, 'name':'T1'},
    1 :{'x':1100.0, 'y': 480, 'z':0.0, 'D':D, 'zhub':zhub, 'cmax':cmax, 'fmax':fmax, 'Cmeander':Cmeander, 'name':'T2'},
    2 :{'x':1800.0, 'y': 480, 'z':0.0, 'D':D, 'zhub':zhub, 'cmax':cmax, 'fmax':fmax, 'Cmeander':Cmeander, 'name':'T3'},
    3 :{'x': 500.0, 'y':1200, 'z':0.0, 'D':D, 'zhub':zhub, 'cmax':cmax, 'fmax':fmax, 'Cmeander':Cmeander, 'name':'T4'},
    4 :{'x':1100.0, 'y':1200, 'z':0.0, 'D':D, 'zhub':zhub, 'cmax':cmax, 'fmax':fmax, 'Cmeander':Cmeander, 'name':'T5'},
    5 :{'x':1800.0, 'y':1200, 'z':0.0, 'D':D, 'zhub':zhub, 'cmax':cmax, 'fmax':fmax, 'Cmeander':Cmeander, 'name':'T6'},
}
refTurb_rot = 0

# ----------- Turbine parameters
# Set the yaw of each turbine for wind dir. One row for each wind direction.
yaw_init = None


# -----------------------------------------------------------------------------
# ------------------- Inflow conditions and input files -----------------------
# -----------------------------------------------------------------------------

# ----------- Additional variables
tmax = 100      # Total simulation time
nSeeds = 6      # Number of seeds
zbot = 1        # Bottom of your domain
mod_wake = 2    # Wake model. 1: Polar, 2: Curled, 3: Cartesian

# ----------- Inflow parameters
inflowType = 'TS'

# ----------- Desired sweeps
vhub       = [8]
shear      = [0.1]
TIvalue    = [10]
inflow_deg = [0]

# ----------- Template files
templatePath = '/projects/tcwnd/rthedin/fastfarm/ff_workshop_nawea2025/tutorial_files'
# Files should be in templatePath. Put None on any input that is not applicable.
templateFiles = {
    "EDfilename"              : 'IEA-15-240-RWT-Monopile_ElastoDyn.T',
    'SrvDfilename'            : 'IEA-15-240-RWT-Monopile_ServoDyn.T',
    'ADfilename'              : 'IEA-15-240-RWT-Monopile_AeroDyn15.dat',
    'ADbladefilename'         : 'IEA-15-240-RWT_AeroDyn15_blade.dat',
    'IWfilename'              : 'IW_WT.dat',
    'EDbladefilename'         : 'IEA-15-240-RWT_ElastoDyn_blade.dat',
    'EDtowerfilename'         : 'IEA-15-240-RWT-Monopile_ElastoDyn_tower.dat',
    'turbfilename'            : 'IEA-15-240-RWT-Monopile.T',
    'libdisconfilepath'       : '/projects/tcwnd/rthedin/repos/ROSCO_v2.9.5/rosco/controller/build/libdiscon.so',
    'controllerInputfilename' : 'IEA-15-240-RWT-Monopile_DISCON_ROSCOv2.9.IN',
    'FFfilename'              : 'FAST.Farm.fstf',
    'turbsimLowfilepath'      : '/home/rthedin/repos/openfast_toolbox/openfast_toolbox/fastfarm/examples/SampleFiles/template_Low_InflowXX_SeedY.inp',
    'turbsimHighfilepath'     : '/home/rthedin/repos/openfast_toolbox/openfast_toolbox/fastfarm/examples/SampleFiles/template_HighT1_InflowXX_SeedY.inp'
}
# SLURM scripts
slurm_TS_high           = '/home/rthedin/repos/openfast_toolbox/openfast_toolbox/fastfarm/examples/SampleFiles/runAllHighBox.sh'
slurm_TS_low            = '/home/rthedin/repos/openfast_toolbox/openfast_toolbox/fastfarm/examples/SampleFiles/runAllLowBox.sh'
slurm_FF_single         = '/home/rthedin/repos/openfast_toolbox/openfast_toolbox/fastfarm/examples/SampleFiles/runFASTFarm_cond0_case0_seed0.sh'


# -----------------------------------------------------------------------------
# ---------------------- Discretization parameters ----------------------------
# -----------------------------------------------------------------------------
# Let's start with no knowledge of what resolution we should use.

# ----------- Low- and high-res boxes parameters
# High-res boxes settings
dt_high     = None   # sampling frequency of high-res files
ds_high     = None   # dx, dy, dz of high-res files
extent_high = None   # extent in y and x for each turbine, in D
# Low-res boxes settings
dt_low      = None   # sampling frequency of low-res files
ds_low      = None   # dx, dy, dz of low-res files
extent_low  = None   # extent in [xmin,xmax,ymin,ymax,zmax], in D


# -----------------------------------------------------------------------------
# END OF USER INPUT
# -----------------------------------------------------------------------------

In [ ]:
# -----------------------------------------------------------------------------
# -------------------- FAST.Farm initial setup --------------------------------
# -----------------------------------------------------------------------------
# ----------- Initial setup
# The initial setup with the temporal and spatial resolutions of both high and
# low resolution boxes given as None will trigger their automatic computation.
amr = FFCaseCreation(path, wts, tmax, zbot, vhub, shear, TIvalue, inflow_deg,
                     dt_high=dt_high, ds_high=ds_high, extent_high=extent_high,
                     dt_low=dt_low,   ds_low=ds_low,   extent_low=extent_low,
                     ffbin=ffbin, mod_wake=mod_wake, yaw_init=yaw_init,
                     nSeeds=nSeeds, tsbin=tsbin, inflowType=inflowType,
                     refTurb_rot=refTurb_rot, verbose=0)

The call to `FFCaseCreation` with all the resolutions as `None` triggers the lengthy message above. From there, we can just check if those numbers make sense given the available computational resources. Highlight the warning printed with further instructions. Also, note that `dt_high=0.3` is a just a consequence of the value I put as `fmax`, which isn't the actual maximum exitation frequency of the IEA 15MW. It is slightly more conservative than the (already conservative) 12P excitation frequency of about 1.5Hz. The reason I chose 10/6 is to get a round number for dt_high.

In this case, it is saying that the low-res should be 24.32, but since it needs to be a multiple of the high res, then it automatically selects 20 m. However, since 24.32 is close to 25, sometimes we might want to just run at 25. Then, an option is to re-run the same command, but now passing such a value (see warning above):

In [ ]:
ds_low      = 25

# -----------------------------------------------------------------------------
# -------------------- FAST.Farm initial setup --------------------------------
# -----------------------------------------------------------------------------
# ----------- Initial setup
# The initial setup with the temporal and spatial resolutions of both high and
# low resolution boxes given as None will trigger their automatic computation.
amr = FFCaseCreation(path, wts, tmax, zbot, vhub, shear, TIvalue, inflow_deg,
                     dt_high=dt_high, ds_high=ds_high, extent_high=extent_high,
                     dt_low=dt_low,   ds_low=ds_low,   extent_low=extent_low,
                     ffbin=ffbin, mod_wake=mod_wake, yaw_init=yaw_init,
                     nSeeds=nSeeds, tsbin=tsbin, inflowType=inflowType,
                     refTurb_rot=refTurb_rot, verbose=0)

Note how now the low-res grid resolution is set to 25 m. We can do the same with the dt if we want. Let's say we want the time steps of the low to be a more round number, we can set both dts:

In [ ]:
dt_high     = 0.25
dt_low      = 1.00 
ds_low      = 25

# -----------------------------------------------------------------------------
# -------------------- FAST.Farm initial setup --------------------------------
# -----------------------------------------------------------------------------
# ----------- Initial setup
# The initial setup with the temporal and spatial resolutions of both high and
# low resolution boxes given as None will trigger their automatic computation.
amr = FFCaseCreation(path, wts, tmax, zbot, vhub, shear, TIvalue, inflow_deg,
                     dt_high=dt_high, ds_high=ds_high, extent_high=extent_high,
                     dt_low=dt_low,   ds_low=ds_low,   extent_low=extent_low,
                     ffbin=ffbin, mod_wake=mod_wake, yaw_init=yaw_init,
                     nSeeds=nSeeds, tsbin=tsbin, inflowType=inflowType,
                     refTurb_rot=refTurb_rot, verbose=0)

This ends the illustration of the first example. Now we can move forward with the FAST.Farm setup using two options:

1. Use directly the `amr` object:
```
# ----------- Low- and high-res boxes parameters
# High-res boxes settings
dt_high     = amr.dt_high
ds_high     = amr.ds_high
extent_high = amr.extent_high
# Low-res boxes settings
dt_low      = amr.dt_low
ds_low      = amr.ds_low
extent_low  = amr.extent_low
```

2. Manually add those values to their corresponding variables:
```
# ----------- Low- and high-res boxes parameters
# High-res boxes settings
dt_high     =  0.25               
ds_high     =  5                 
extent_high =  1.2               
# Low-res boxes settings
dt_low      = 1.0                  
ds_low      = 25                 
extent_low  = [1.5,2.5,1.5,1.5,2]

```

In [ ]:
# Note you can always print the object and get some information about the farm and the set of cases that will be setup:
print(amr)

# Example 2: `Ex2a_FASTFarm_TurbSim_driven.py`

The example 2 for TurbSim has the very same initial setup. The cell below is a copy of the first cell of _Example 1_, except that the discretization values are now populated

In [ ]:
# -----------------------------------------------------------------------------
# USER INPUT: Modify these
# -----------------------------------------------------------------------------

# -----------------------------------------------------------------------------
# ------------------------ General parameters ---------------------------------
# -----------------------------------------------------------------------------

# ----------- Case absolute path
path = '/projects/tcwnd/rthedin/fastfarm/ff_workshop_nawea2025/ff_examples_ex2a'

# ----------- Execution parameters
# If you are sure the correct binaries are first on your $PATH and associated
# libraries on $LD_LIBRARY_PATH, you can set the variables below to None or
# remove them from the `FFCaseCreation` call
ffbin = '/projects/tcwnd/rthedin/repos/openfast_v4.1.1/build/install/bin/FAST.Farm'
tsbin = '/projects/tcwnd/rthedin/repos/openfast_v4.1.1/build/install/bin/turbsim'


# -----------------------------------------------------------------------------
# --------------------------- Farm parameters ---------------------------------
# -----------------------------------------------------------------------------

# ----------- General turbine parameters
cmax     = 5      # Maximum blade chord (m)
fmax     = 10/6   # Maximum excitation frequency (Hz)
Cmeander = 1.9    # Meandering constant (-)
D = 240           # Rotor diameter (m)
zhub = 150        # Hub height (m)

# ----------- Wind farm
# The wts dictionary holds information of each wind turbine. The allowed entries
# are: x, y, z, D, zhub, cmax, fmax, Cmeander, and phi_deg. The phi_deg is the
# only entry that is optional and is related to floating platform heading angle,
# given in degrees. The angle phi_deg is not illustrated on the example below.
wts = {
    0 :{'x': 500.0, 'y': 480, 'z':0.0, 'D':D, 'zhub':zhub, 'cmax':cmax, 'fmax':fmax, 'Cmeander':Cmeander, 'name':'T1'},
    1 :{'x':1100.0, 'y': 480, 'z':0.0, 'D':D, 'zhub':zhub, 'cmax':cmax, 'fmax':fmax, 'Cmeander':Cmeander, 'name':'T2'},
    2 :{'x':1800.0, 'y': 480, 'z':0.0, 'D':D, 'zhub':zhub, 'cmax':cmax, 'fmax':fmax, 'Cmeander':Cmeander, 'name':'T3'},
    3 :{'x': 500.0, 'y':1200, 'z':0.0, 'D':D, 'zhub':zhub, 'cmax':cmax, 'fmax':fmax, 'Cmeander':Cmeander, 'name':'T4'},
    4 :{'x':1100.0, 'y':1200, 'z':0.0, 'D':D, 'zhub':zhub, 'cmax':cmax, 'fmax':fmax, 'Cmeander':Cmeander, 'name':'T5'},
    5 :{'x':1800.0, 'y':1200, 'z':0.0, 'D':D, 'zhub':zhub, 'cmax':cmax, 'fmax':fmax, 'Cmeander':Cmeander, 'name':'T6'},
}
refTurb_rot = 0

# ----------- Turbine parameters
# Set the yaw of each turbine for wind dir. One row for each wind direction.
yaw_init = None


# -----------------------------------------------------------------------------
# ------------------- Inflow conditions and input files -----------------------
# -----------------------------------------------------------------------------

# ----------- Additional variables
tmax = 50      # Total simulation time
nSeeds = 6      # Number of seeds
zbot = 1        # Bottom of your domain
mod_wake = 2    # Wake model. 1: Polar, 2: Curled, 3: Cartesian

# ----------- Inflow parameters
inflowType = 'TS'

# ----------- Desired sweeps
vhub       = [8]
shear      = [0.1]
TIvalue    = [10]
inflow_deg = [0]

# ----------- Template files
templatePath = '/projects/tcwnd/rthedin/fastfarm/ff_workshop_nawea2025/tutorial_files'
# Files should be in templatePath. Put None on any input that is not applicable.
templateFiles = {
    "EDfilename"              : 'IEA-15-240-RWT-Monopile_ElastoDyn.T',
    'SrvDfilename'            : 'IEA-15-240-RWT-Monopile_ServoDyn.T',
    'ADfilename'              : 'IEA-15-240-RWT-Monopile_AeroDyn15.dat',
    'ADbladefilename'         : 'IEA-15-240-RWT_AeroDyn15_blade.dat',
    'IWfilename'              : 'IW_WT.dat',
    'EDbladefilename'         : 'IEA-15-240-RWT_ElastoDyn_blade.dat',
    'EDtowerfilename'         : 'IEA-15-240-RWT-Monopile_ElastoDyn_tower.dat',
    'turbfilename'            : 'IEA-15-240-RWT-Monopile.T',
    'libdisconfilepath'       : '/projects/tcwnd/rthedin/repos/ROSCO_v2.9.5/rosco/controller/build/libdiscon.so',
    'controllerInputfilename' : 'IEA-15-240-RWT-Monopile_DISCON_ROSCOv2.9.IN',
    'FFfilename'              : 'FAST.Farm.fstf',
    'turbsimLowfilepath'      : '/home/rthedin/repos/openfast_toolbox/openfast_toolbox/fastfarm/examples/SampleFiles/template_Low_InflowXX_SeedY.inp',
    'turbsimHighfilepath'     : '/home/rthedin/repos/openfast_toolbox/openfast_toolbox/fastfarm/examples/SampleFiles/template_HighT1_InflowXX_SeedY.inp'
}
# SLURM scripts
slurm_TS_high           = '/home/rthedin/repos/openfast_toolbox/openfast_toolbox/fastfarm/examples/SampleFiles/runAllHighBox.sh'
slurm_TS_low            = '/home/rthedin/repos/openfast_toolbox/openfast_toolbox/fastfarm/examples/SampleFiles/runAllLowBox.sh'
slurm_FF_single         = '/home/rthedin/repos/openfast_toolbox/openfast_toolbox/fastfarm/examples/SampleFiles/runFASTFarm_cond0_case0_seed0.sh'


# -----------------------------------------------------------------------------
# ---------------------- Discretization parameters ----------------------------
# -----------------------------------------------------------------------------
# ----------- Low- and high-res boxes parameters
# High-res boxes settings
dt_high     =  0.25               # sampling frequency of high-res files
ds_high     =  5                  # dx, dy, dz of high-res files
extent_high =  1.2                # extent in y and x for each turbine, in D
# Low-res boxes settings
dt_low      = 1.0                 # sampling frequency of low-res files
ds_low      = 25                  # dx, dy, dz of low-res files
extent_low  = [1.5,2.5,1.5,1.5,2] # extent in [xmin,xmax,ymin,ymax,zmax], in D


# -----------------------------------------------------------------------------
# END OF USER INPUT
# -----------------------------------------------------------------------------

(Setting verbose to 1 on the call below:)

In [ ]:
# -----------------------------------------------------------------------------
# -------------------- FAST.Farm initial setup --------------------------------
# -----------------------------------------------------------------------------
# ----------- Initial setup
ffcase = FFCaseCreation(path, wts, tmax, zbot, vhub, shear, TIvalue, inflow_deg,
                        dt_high=dt_high, ds_high=ds_high, extent_high=extent_high,
                        dt_low=dt_low,   ds_low=ds_low,   extent_low=extent_low,
                        ffbin=ffbin, mod_wake=mod_wake, yaw_init=yaw_init,
                        nSeeds=nSeeds, tsbin=tsbin, inflowType=inflowType,
                        refTurb_rot=refTurb_rot, verbose=1)

# ----------- Perform auxiliary steps in preparing the case
ffcase.setTemplateFilename(templatePath, templateFiles)
ffcase.getDomainParameters()
ffcase.copyTurbineFilesForEachCase()

There are a few things the user can look at to check the setup. One of them, as mentioned is to print the object. Others are to print the datasets that hold the information for all the cases:

In [ ]:
ffcase.allCases

You have easy access to information related to your case. This FAST.Farm setup is for a single condition with a single case, so this array is simple. Will show later more complex cases and how to pick out information from the array above.

You can also plot your setup. On the plot, the extents set previously become clear. See how the `extent_low` related to xmax is larger than the others.

In [ ]:
ffcase.plot()  # add showTurbNumber=True to help with potential debugging
# The plot command above saves the images to the case root `ffcase.path`

Now comes the main part, which is the turbsim setup. The third call is commented out since you won't be doing this from an HPC.

In [ ]:
# -----------------------------------------------------------------------------
# ---------------------- TurbSim setup and execution --------------------------
# -----------------------------------------------------------------------------
# ----------- TurbSim low-res setup
ffcase.TS_low_setup()
# ----------- Prepare script for submission
ffcase.TS_low_slurm_prepare(slurm_TS_low)
# ----------- Submit the low-res script (can be done from the command line)
#ffcase.TS_low_slurm_submit()

Now, if you go the the directory of this case, `ffcase.path`, you will see a _condition_ directory, the png of the farm above, and a bash script to be submitted to slurm. You can submit it through the command line or you can use the `ffcase.TS_low_slurm_submit()` above. You should open the script and show people what it is doing. Some stuff there (like modules) is given for Kestrel. For someone to use it on their own HPC system, they just need to create new template versions of those and give as inputs through the `slurm_TS_high`, `slurm_TS_low`, and `slurm_FF_single` variables.

In [ ]:
 # For the sake of illustration here, I'll submit it. Note this calls takes a few key SLURM parameters. (takes about 10s)
ffcase.TS_low_slurm_submit(A='shellwind', qos='high', t='0:10:00', p='debug')

Now with the low-res completed, we can move forward with the high-res. Note that if you try to call `ffcase.TS_high_setup()` before the low-res is done, it will stop when trying to open the low-res box.

In [ ]:
# ----------- TurbSim high-res setup
ffcase.TS_high_setup()
# ----------- Prepare script for submission
ffcase.TS_high_slurm_prepare(slurm_TS_high)
# ----------- Submit the high-res script (can be done from the command line)
#ffcase.TS_high_slurm_submit()

The calls above create the time series to drive the high-res box, taking into account where the turbines are, inclusing the `offset` parameter to TurbSim and also the rolling of the time series, considering how downstream each turbine is.

In [ ]:
# Let's run the turbsim boxes. Takes about 3min at the 0.25s temporal res
ffcase.TS_high_slurm_submit(A='shellwind', qos='high', t='0:06:00', p='debug')

After this runs, we can move forward with the FAST.Farm setup. Note how the wake model parameters can be easily set.

In [ ]:
# -----------------------------------------------------------------------------
# ------------------ Finish FAST.Farm setup and execution ---------------------
# -----------------------------------------------------------------------------
# ----------- FAST.Farm setup
ffcase.FF_setup()
# Update wake model constants (adjust as needed for your turbine model)
ffcase.set_wake_model_params(k_VortexDecay=0, k_vCurl=2.8)

# ----------- Prepare script for submission
ffcase.FF_slurm_prepare(slurm_FF_single)

# ----------- Submit the FAST.Farm script (can be done from the command line)
#ffcase.FF_slurm_submit(p='debug', t='1:00:00')

In [ ]:
# Modify the time step and UA model for this specific case. This also illustrates how to change variables
import os
from openfast_toolbox.fastfarm.FASTFarmCaseCreation import modifyProperty

for cond in range(ffcase.nConditions):
    for case in range(ffcase.nCases):
        ad_file  = os.path.join(ffcase.path, ffcase.condDirList[cond], ffcase.caseDirList[case], ffcase.ADfilename)
        modifyProperty(ad_file, 'UA_Mod', 4)
        modifyProperty(ad_file, 'SectAvg', False)
        for turb in range(ffcase.nTurbines):
            fst_file = os.path.join(ffcase.path, ffcase.condDirList[cond], ffcase.caseDirList[case], f'{ffcase.turbfilename}{turb+1}.fst')
            modifyProperty(fst_file, 'DT', 0.001)
            

In [ ]:
# We have 6 seeds, so 6 cases. This call will submit each one of them separately. Should take ~5min each.
ffcase.FF_slurm_submit(A='shellwind', qos='high', t='1:00:00', p='debug')

In [ ]:
# We can finally just dump the object to disk so we can re-open another time or for post-processing. It will be saved on the ffcase.path path
ffcase.save()

This concludes the second example. If the input files were valid (meaning they would not result in a fatal error using OpenFAST), the FAST.Farm simulation would likely be successful here.

# 3. More complex setups

# 3.1 Wind speed and wind dir sweep

Let's now try to sweep in some wind speeds and directions.

In [ ]:
# -----------------------------------------------------------------------------
# USER INPUT: Modify these
# -----------------------------------------------------------------------------

# -----------------------------------------------------------------------------
# ------------------------ General parameters ---------------------------------
# -----------------------------------------------------------------------------

# ----------- Case absolute path
path = '/projects/tcwnd/rthedin/fastfarm/ff_workshop_nawea2025/ff_examples_ex2a_widr'

# ----------- Execution parameters
# If you are sure the correct binaries are first on your $PATH and associated
# libraries on $LD_LIBRARY_PATH, you can set the variables below to None or
# remove them from the `FFCaseCreation` call
ffbin = '/projects/tcwnd/rthedin/repos/openfast_v4.1.1/build/install/bin/FAST.Farm'
tsbin = '/projects/tcwnd/rthedin/repos/openfast_v4.1.1/build/install/bin/turbsim'


# -----------------------------------------------------------------------------
# --------------------------- Farm parameters ---------------------------------
# -----------------------------------------------------------------------------

# ----------- General turbine parameters
cmax     = 5      # Maximum blade chord (m)
fmax     = 10/6   # Maximum excitation frequency (Hz)
Cmeander = 1.9    # Meandering constant (-)
D = 240           # Rotor diameter (m)
zhub = 150        # Hub height (m)

# ----------- Wind farm
# The wts dictionary holds information of each wind turbine. The allowed entries
# are: x, y, z, D, zhub, cmax, fmax, Cmeander, and phi_deg. The phi_deg is the
# only entry that is optional and is related to floating platform heading angle,
# given in degrees. The angle phi_deg is not illustrated on the example below.
wts = {
    0 :{'x': 500.0, 'y': 480, 'z':0.0, 'D':D, 'zhub':zhub, 'cmax':cmax, 'fmax':fmax, 'Cmeander':Cmeander, 'name':'T1'},
    1 :{'x':1100.0, 'y': 480, 'z':0.0, 'D':D, 'zhub':zhub, 'cmax':cmax, 'fmax':fmax, 'Cmeander':Cmeander, 'name':'T2'},
    2 :{'x':1800.0, 'y': 480, 'z':0.0, 'D':D, 'zhub':zhub, 'cmax':cmax, 'fmax':fmax, 'Cmeander':Cmeander, 'name':'T3'},
    3 :{'x': 500.0, 'y':1200, 'z':0.0, 'D':D, 'zhub':zhub, 'cmax':cmax, 'fmax':fmax, 'Cmeander':Cmeander, 'name':'T4'},
    4 :{'x':1100.0, 'y':1200, 'z':0.0, 'D':D, 'zhub':zhub, 'cmax':cmax, 'fmax':fmax, 'Cmeander':Cmeander, 'name':'T5'},
    5 :{'x':1800.0, 'y':1200, 'z':0.0, 'D':D, 'zhub':zhub, 'cmax':cmax, 'fmax':fmax, 'Cmeander':Cmeander, 'name':'T6'},
}
refTurb_rot = 0

# ----------- Turbine parameters
# Set the yaw of each turbine for wind dir. One row for each wind direction.
yaw_init = None


# -----------------------------------------------------------------------------
# ------------------- Inflow conditions and input files -----------------------
# -----------------------------------------------------------------------------

# ----------- Additional variables
tmax = 100      # Total simulation time
nSeeds = 6      # Number of seeds
zbot = 1        # Bottom of your domain
mod_wake = 2    # Wake model. 1: Polar, 2: Curled, 3: Cartesian

# ----------- Inflow parameters
inflowType = 'TS'

# ----------- Desired sweeps
vhub       = [8, 10]
shear      = [0.1]
TIvalue    = [10]
inflow_deg = [-5, 0, 5]


# ----------- Template files
templatePath = '/projects/tcwnd/rthedin/fastfarm/ff_workshop_nawea2025/tutorial_files'
# Files should be in templatePath. Put None on any input that is not applicable.
templateFiles = {
    "EDfilename"              : 'IEA-15-240-RWT-Monopile_ElastoDyn.T',
    'SrvDfilename'            : 'IEA-15-240-RWT-Monopile_ServoDyn.T',
    'ADfilename'              : 'IEA-15-240-RWT-Monopile_AeroDyn15.dat',
    'ADbladefilename'         : 'IEA-15-240-RWT_AeroDyn15_blade.dat',
    'IWfilename'              : 'IW_WT.dat',
    'EDbladefilename'         : 'IEA-15-240-RWT_ElastoDyn_blade.dat',
    'EDtowerfilename'         : 'IEA-15-240-RWT-Monopile_ElastoDyn_tower.dat',
    'turbfilename'            : 'IEA-15-240-RWT-Monopile.T',
    'libdisconfilepath'       : '/projects/tcwnd/rthedin/repos/ROSCO_v2.9.5/rosco/controller/build/libdiscon.so',
    'controllerInputfilename' : 'IEA-15-240-RWT-Monopile_DISCON_ROSCOv2.9.IN',
    'FFfilename'              : 'FAST.Farm.fstf',
    'turbsimLowfilepath'      : '/home/rthedin/repos/openfast_toolbox/openfast_toolbox/fastfarm/examples/SampleFiles/template_Low_InflowXX_SeedY.inp',
    'turbsimHighfilepath'     : '/home/rthedin/repos/openfast_toolbox/openfast_toolbox/fastfarm/examples/SampleFiles/template_HighT1_InflowXX_SeedY.inp'
}
# SLURM scripts
slurm_TS_high           = '/home/rthedin/repos/openfast_toolbox/openfast_toolbox/fastfarm/examples/SampleFiles/runAllHighBox.sh'
slurm_TS_low            = '/home/rthedin/repos/openfast_toolbox/openfast_toolbox/fastfarm/examples/SampleFiles/runAllLowBox.sh'
slurm_FF_single         = '/home/rthedin/repos/openfast_toolbox/openfast_toolbox/fastfarm/examples/SampleFiles/runFASTFarm_cond0_case0_seed0.sh'


# -----------------------------------------------------------------------------
# ---------------------- Discretization parameters ----------------------------
# -----------------------------------------------------------------------------
# ----------- Low- and high-res boxes parameters
# High-res boxes settings
dt_high     =  0.25               # sampling frequency of high-res files
ds_high     =  5                  # dx, dy, dz of high-res files
extent_high =  1.2                # extent in y and x for each turbine, in D
# Low-res boxes settings
dt_low      = 1.0                 # sampling frequency of low-res files
ds_low      = 25                  # dx, dy, dz of low-res files
extent_low  = [1.5,2.5,1.5,1.5,2] # extent in [xmin,xmax,ymin,ymax,zmax], in D


# -----------------------------------------------------------------------------
# END OF USER INPUT
# -----------------------------------------------------------------------------

In [ ]:
# -----------------------------------------------------------------------------
# -------------------- FAST.Farm initial setup --------------------------------
# -----------------------------------------------------------------------------
# ----------- Initial setup
ffcase = FFCaseCreation(path, wts, tmax, zbot, vhub, shear, TIvalue, inflow_deg,
                        dt_high=dt_high, ds_high=ds_high, extent_high=extent_high,
                        dt_low=dt_low,   ds_low=ds_low,   extent_low=extent_low,
                        ffbin=ffbin, mod_wake=mod_wake, yaw_init=yaw_init,
                        nSeeds=nSeeds, tsbin=tsbin, inflowType=inflowType,
                        refTurb_rot=refTurb_rot, verbose=1)

# ----------- Perform auxiliary steps in preparing the case
ffcase.setTemplateFilename(templatePath, templateFiles)
ffcase.getDomainParameters()
ffcase.copyTurbineFilesForEachCase()

Note that we asked for 2 wind speeds (two _conditions_) and 3 inflow directions. A change in direction only doesn't mean we will re-run the inflow, but rather we will rotate the farm. So a sweep in wind direction means another _case_. For each _condition_, we then have 3 _cases_. We can see all the conditions and cases as follows:

In [ ]:
ffcase.allCond

In [ ]:
ffcase.allCases

In [ ]:
# Now the plotting function is more helpful. See how the farm was rotated to accomodate the new wind direction
# Also note how the turbine has been yawed so that it remains aligned given the change in the wind direction.
ffcase.plot()

# 3.2 Yaw misalignment

Let's add some yaw misalignment. Let's reduce to a single _condition_ and reduce the sweeps in wdir to only two.

In [ ]:
# ----------- Case absolute path
path = '/projects/tcwnd/rthedin/fastfarm/ff_workshop_nawea2025/ff_examples_ex2a_yaw'

# ----------- Turbine parameters
# Set the yaw of each turbine for wind dir. One row for each wind direction.
yaw_init = [[0, 0, 0, 0, 0, 0],
            [8, 8, 8, 8, 8, 8]]

# ----------- Desired sweeps
vhub       = [10]
inflow_deg = [0, 5]


In [ ]:
# -----------------------------------------------------------------------------
# -------------------- FAST.Farm initial setup --------------------------------
# -----------------------------------------------------------------------------
# ----------- Initial setup
ffcase = FFCaseCreation(path, wts, tmax, zbot, vhub, shear, TIvalue, inflow_deg,
                        dt_high=dt_high, ds_high=ds_high, extent_high=extent_high,
                        dt_low=dt_low,   ds_low=ds_low,   extent_low=extent_low,
                        ffbin=ffbin, mod_wake=mod_wake, yaw_init=yaw_init,
                        nSeeds=nSeeds, tsbin=tsbin, inflowType=inflowType,
                        refTurb_rot=refTurb_rot, verbose=1)

# ----------- Perform auxiliary steps in preparing the case
ffcase.setTemplateFilename(templatePath, templateFiles)
ffcase.getDomainParameters()
ffcase.copyTurbineFilesForEachCase()

Note, however, that the setup above does zero yaw with the 0 wind direction and 8 deg yaw with the 5 deg wind direction. These fields are not combinatory. If we want that, we need to set it up slightly different:

In [ ]:
# ----------- Case absolute path
path = '/projects/tcwnd/rthedin/fastfarm/ff_workshop_nawea2025/ff_examples_ex2a_yaw2'

# ----------- Turbine parameters
# Set the yaw of each turbine for wind dir. One row for each wind direction.
yaw_init = [[0, 0, 0, 0, 0, 0],
            [8, 8, 8, 8, 8, 8],
            [0, 0, 0, 0, 0, 0],
            [8, 8, 8, 8, 8, 8]]

# ----------- Desired sweeps
vhub       = [10]
inflow_deg = [0, 0, 5, 5]


In [ ]:
# -----------------------------------------------------------------------------
# -------------------- FAST.Farm initial setup --------------------------------
# -----------------------------------------------------------------------------
# ----------- Initial setup
ffcase = FFCaseCreation(path, wts, tmax, zbot, vhub, shear, TIvalue, inflow_deg,
                        dt_high=dt_high, ds_high=ds_high, extent_high=extent_high,
                        dt_low=dt_low,   ds_low=ds_low,   extent_low=extent_low,
                        ffbin=ffbin, mod_wake=mod_wake, yaw_init=yaw_init,
                        nSeeds=nSeeds, tsbin=tsbin, inflowType=inflowType,
                        refTurb_rot=refTurb_rot, verbose=1)

# ----------- Perform auxiliary steps in preparing the case
ffcase.setTemplateFilename(templatePath, templateFiles)
ffcase.getDomainParameters()
ffcase.copyTurbineFilesForEachCase()

In [ ]:
ffcase.allCases

In [ ]:
# Note how each turbine has two yaw angles now (less confusing to see on T6)
ffcase.plot()

# 4. Some details about the case setups and some bonus things you can do with the toolbox

## 4.1. LES sampling

In [1]:
# to do

## 4.2. Changing specific variables on all cases

In [ ]:
# ----------- Wake planes to be saved for each turbine (only 9 planes at most can be saved)
# The offset is useful if we start sampling at 0 D, so that we actually have an usable plane that is not _at_ the hub plane. Here we are not interested
wakeplanes_in_D = [3]
offset = 0
# Wake points (not planes). Can have up to 9 per turbine
wakepoints_in_D = [1,2,3,4,5,5.9]


# Add wake plane saving (the hub-height and vertical thought the centerline planes are already properly configured though `FF_setup()`
for cond in range(ffcase.nConditions):
    for case in range(ffcase.nCases):
        for seed in range(ffcase.nSeeds):
            ff_file = os.path.join(ffcase.path, ffcase.condDirList[cond], ffcase.caseDirList[case], f'Seed_{seed}', ffcase.outputFFfilename)
            # Parameters for the wake locations
            Tx = ffcase.allCases.isel(case=case).Tx.values
            # Wake plane (yz plane)
            outdistYZ = np.array([[w*ffcase.D for w in wakeplanes_in_D]+tx+offset for tx in Tx]).flatten()
            outdiststrYZ = ', '.join(map(str, outdistYZ))
            modifyProperty(ff_file, 'NOutDisWindYZ', len(outdistYZ))
            modifyProperty(ff_file, 'OutDisWindX', outdiststrYZ)
            # Wake (repeated for every turbine within FAST.Farm)
            outdist = np.array([w*ffcase.D+offset for w in wakepoints_in_D]).flatten()
            outdiststr = ', '.join(map(str, outdist))
            modifyProperty(ff_file, 'NOutDist', len(outdist))
            modifyProperty(ff_file, 'OutDist', outdiststr)
            # Let's change the frequency planes are saved (default is dt_low)
            modifyProperty(ff_file, 'WrDisDT', 1)


# 5. Some post-processing

In [ ]:
# to do